In [2]:
from itertools import count
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    length = lookup(entry, 'Total Time')
    rating = lookup(entry, 'Rating')
    count = lookup(entry, 'Play Count')
    genre = lookup(entry, 'Genre')

    if name is None or artist is None or album is None : 
        continue

    print(name, artist, album, genre)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
    VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()


Dict count: 404
Another One Bites The Dust Queen Greatest Hits Rock
Asche Zu Asche Rammstein Herzeleid Industrial
Beauty School Dropout Various Grease Soundtrack
Black Dog Led Zeppelin IV Rock
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] Rock
Circles Bryan Lee Blues Is Funk
Comfortably Numb Pink Floyd The Wall [Disc 2] Rock
Crazy Little Thing Called Love Queen Greatest Hits Rock
Electric Funeral Black Sabbath Paranoid Metal
Fat Bottomed Girls Queen Greatest Hits Rock
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock
Four Sticks Led Zeppelin IV Rock
Furious Angels Rob Dougan The Matrix Reloaded Soundtrack
Gelle Bryan Lee Blues Is Blues/R&B
Going To California Led Zeppelin IV Rock
Grease Various Grease Soundtrack
Hand of Doom Black Sabbath Paranoid Metal
Hells Bells AC/DC Who Made Who Rock
Hey You Pink Floyd The Wall [Disc 2] Rock
I Worry Bryan Lee Blues Is Blues/R&B
Iron Man Black Sabbath Paranoid Metal
Is There Anybody Out There? Pink Floyd The Wall [Disc 2] Rock

TypeError: 'NoneType' object is not subscriptable